In [2]:
import pandas as pd

In [3]:
def import_data(filename, sheet_count):
    """
    function to import and concatenate excel data
    requires that exported data contains columns:
        index, test_time, step_time, voltage, current, capacity, scapacity, energy, senergy, and state
    """
    names = ['index', 'test_time', 'step_time', 'voltage', 'current', 'capacity', 'scapacity', 'energy', 'senergy', 'state']
    df = pd.read_excel(filename, sheet_name='Sheet1', header=None, names=names)
    dfl = list([df])
    if sheet_count == 1:
        pass
    else:
        for i in range(1, sheet_count):
            sheet_name='Sheet1(Continued{})'.format(i)
            df0 = pd.read_excel(filename, sheet_name, header=None, names=names)
            dfl.append(df0)
    dataframe = pd.concat(dfl, ignore_index=True)
    return dataframe

def test_import_data():
    """
    docstring
    """
    dataframe = import_data('example_data.xls', 2)
    assert isinstance(dataframe, pd.core.frame.DataFrame), 'output is not a pandas dataframe'
    assert len(dataframe) == 130571, 'output should have length 130571, not {}'.format(len(dataframe))
    assert len(dataframe.keys()) == 10, 'output should have 10 keys, not {}'.format(len(dataframe.keys()))

In [4]:
test_import_data()

In [5]:
dataframe = import_data('example_data.xls', 2)
dataframe

,index,test_time,step_time,voltage,current,capacity,scapacity,energy,senergy,state
0,Index,Mode,Period,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Rest,06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Index,TestTime,StepTime,Voltage/V,Current/mA,Capacity/mAh,SCapacity/mAh/g,Energy/mWh,SEnergy/Wh/kg,State
3,1,00:00:00,00:00:00,2.346,0,0,0,0,0,R
4,2,00:10:00,00:10:00,2.3449,0,0,0,0,0,R
5,3,00:20:00,00:20:00,2.3449,0,0,0,0,0,R
6,4,00:30:00,00:30:00,2.3441,0,0,0,0,0,R
7,5,00:40:00,00:40:00,2.3445,0,0,0,0,0,R
8,6,00:50:00,00:50:00,2.3441,0,0,0,0,0,R
9,7,01:00:00,01:00:00,2.3445,0,0,0,0,0,R


In [6]:
def clean_prep_break(dataframe):
    """
    function that removes labels from in between cycles,
    adds a new column which contains the time in seconds,
    and outputs a list of cycle_breaks
    """
#     drop = ['Mode', 'Rest', 'Charge CC', 'Discharge CC', 'TestTime']
    times = dataframe['test_time']
    drop_index = []
    index = []
    for i, _ in enumerate(times):
        time_0 = times[i]
        if isinstance(time_0, str):
            drop_index.append(i)
        else:
            index.append(i)   
    dataframe2 = dataframe.drop(drop_index)
    # add column with time converted to seconds
    t_sec = []
    times = dataframe2['test_time']
    for i in range(len(dataframe2['test_time'])):
        j = index[i]
        time_0 = str(times[j])
        if len(time_0) < 10:
            days = 0
            hours, minutes, seconds = time_0.split(':')
        else:
            days, time = time_0.split('-')
            hours, minutes, seconds = time.split(':')
        sec = int(days)*86400 + int(hours)*3600 + int(minutes)*60 + int(seconds)
        t_sec.append(sec)
    dataframe2['test_time_sec'] = t_sec
    # converts drop_index to a list of cycle_breaks
    cycle_break = []
    for i in range(len(dataframe.index)):
        if i - 1 in drop_index and i + 1 in drop_index:
            cycle_break.append(i)
        else:
            pass
    return dataframe2, cycle_break

In [7]:
dataframe2, cycle_break = clean_prep_break(dataframe)

In [ ]:
dataframe2